In [1]:
import os

while not os.getcwd().endswith("hackathon"):
    os.chdir("..")
    print(f"Now in {os.getcwd()}")

Now in c:\Users\GiovanniGiacometti\Desktop\hackathon


In [2]:
from hackathon.graph.nodes.retrieve import retrieve, filter_with_metadata

INFO:hackathon.managers.vectore_store:Initializing vectorstore...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:hackathon.managers.vectore_store:Vectorstore initialized successfully.


In [3]:
from hackathon.models import MenuMetadata
from hackathon.graph.state import GraphState

In [4]:
from hackathon.session import SessionManager

retriever = SessionManager().vectorstore_manager.retriever

In [13]:
state = GraphState(
    question="What are the most popular dishes in the world?",
    question_id=1,
    metadata=MenuMetadata(
        chef_name="Chef"
    )
)

In [14]:
state.metadata

MenuMetadata(chef_name='Chef', restaurant_name=None, planet_name=None, licences=[])

In [15]:
retriever.invoke(
    "aaaaaa",
    filter=filter_with_metadata(state.metadata),
)

INFO:hackathon.graph.nodes.retrieve:Filtering documents with metadata fields: ['chef_name']


[]

In [16]:
retrieve(state)

INFO:hackathon.graph.nodes.retrieve:Filtering documents with metadata fields: ['chef_name']


{'documents': [Document(id='f4691669-9f1d-4f2c-9caa-2bd214ef9533', metadata={'header_1': 'Menu', 'header_2': 'Portale di Sapori Arcani', 'source': 'L Essenza di Asgard.pdf', 'chef_name': 'Palissandro "Sandro" Luminetti', 'restaurant_name': 'l essenza di asgard', 'planet_name': 'Asgard', 'licences': ['Temporale I', 'Antimateria I', 'Luce I']}, page_content='Lasciate che le vostre papille gustative si immergano in un\'esperienza gastronomica che sfida i limiti del\nconosciuto attraverso il nostro piatto di punta: il "Portale di Sapori Arcani". Questo capolavoro culinario è la\nquintessenza della maestria dello Chef Palissandro Luminetti, una danza sincronizzata tra l\'essenza del\ncosmo e l\'intimità del palato.  \nQuesta straordinaria esperienza culinaria è raccomandata solo per i palati audaci, disposti a esplorare i limiti\ndella percezione umana attraverso un viaggio interdimensionale di gusto. Venite, esploratori del gusto, e\nlasciatevi trasportare in una nuova era di intensità gas

In [3]:
settings_provider = SettingsProvider()

In [4]:
llm = ChatWatsonx(
    model_id=settings_provider.get_ibm_model_name(),  # type: ignore
    url=settings_provider.get_ibm_endpoint_url(),  # type: ignore
    project_id=settings_provider.get_ibm_project_id(),  # type: ignore
)

In [7]:
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

vectorstore = Chroma(
    persist_directory="data/debug_vectorstore",
    embedding_function=HuggingFaceEmbeddings(
        model_name=settings_provider.get_embeddings_model_name(),
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    ),
)

In [23]:
vectorstore.get()["metadatas"][0]

In [31]:
chunk_page_content = "ciao"
chunk_metadata = {"ciao": ["ciao"]}

In [32]:
vectorstore.add_texts(
    texts=[chunk_page_content],
    metadatas=[chunk_metadata],
)

ValueError: Expected metadata value to be a str, int, float or bool, got ['ciao'] which is a list in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [8]:
query_ex = "Quali piatti del ristorante Cosmica Essenza sono preparati utilizzando i Funghi dell’Etere o la tecnica di Marinatura Temporale Sincronizzata?"

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
metadata = {
    
}

retriever.invoke(
    query_ex,

)

In [47]:
import chromadb
from langchain.schema import Document

chroma_client = chromadb.Client()

In [37]:
# collection = chroma_client.create_collection(name="my_collection")

In [46]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id3", "id5"],
    metadatas=[{"fruit": ["pineapple"]}, {"fruit": "orange"}],
)

ValueError: Expected metadata value to be a str, int, float or bool, got ['pineapple'] which is a list in add.

In [34]:


# Example documents
documents = [
    "Document about regression and classification",
    "Document about CNN architecture",
    "Document about both regression and CNN"
]

# Method 1: Using lists in metadata (Recommended)
metadata_list = [
    {
        "techniques": ["regression", "classification"],
        "difficulty": "intermediate"
    },
    {
        "techniques": ["CNN"],
        "difficulty": "advanced"
    },
    {
        "techniques": ["regression", "CNN"],
        "difficulty": "advanced"
    }
]

# Add documents with metadata using lists
collection.add(
    documents=documents,
    ids=["doc1", "doc2", "doc3"],
    metadatas=metadata_list
)

# Query examples
# 1. Find documents about regression
results = collection.query(
    query_texts=["regression"],
    where={"techniques": {"$in": ["regression"]}}
)

# 2. Find advanced documents about CNN
results_advanced_cnn = collection.query(
    query_texts=["CNN architecture"],
    where={
        "$and": [
            {"techniques": {"$in": ["CNN"]}},
            {"difficulty": "advanced"}
        ]
    }
)